# Analyzing a Fortran Stencil Program

## Understanding

Open the <tt>stencil_2d.F90</tt> Fortran program by double clicking on the file in the File Browser on the left (if it is hidden, click on the Folder icon).

![open-stencil2d](img/open-stencil2d.png)

If you prefer, you can also open a new Terminal (File -> New -> Terminal), navigate to the location where <tt>stencil_2d.F90</tt> is located and use your favorite linux editor (e.g. vim) to browse the Fortran source code. This will give you better syntax highlighting.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>1.</b> Read the code and try to understand what the program is doing.<br>
<b>2.</b> There are two global variables of type <tt>integer</tt> named <tt>flop_counter</tt> and <tt>byte_counter</tt>. Change the program to count the number of floating-point operations and bytes transferred to/from memory.<br>
</div>

## Compiling

On Piz Daint, the programming environment is managed using so called modules. We need to load the right modules in order to compile our program. We are going to use the Cray Fortran compiler (<tt>PrgEnv-cray</tt>) for this exercise.

In [13]:
module load daint-gpu
module switch PrgEnv-gnu PrgEnv-cray
module load perftools-lite

ModuleCmd_Switch.c(179):ERROR:152: Module 'PrgEnv-gnu' is currently not loaded


There is a <tt>Makefile</tt> which contains instructions of how to compile our stencil program. <tt>make clean</tt> clean away any previous artefacts from compilation or running our program.

In [14]:
make clean
make

rm -f -rf *~ *.o *.mod *.MOD *.i *.x *.x+orig *.x+[0-9]* core.* *.out
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c m_utils.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF -c stencil_2d.F90
ftn -eZ -ffree -N255 -ec -eC -eI -eF m_utils.o stencil_2d.o -o stencil_2d.x
INFO: creating the CrayPat-instrumented executable 'stencil_2d.x' (lite-samples) ...OK


## Running

We can run our program on all 12 cores of the Xeon E5-2690 v3 Haswell CPU that we have available using the <tt>srun</tt> command. The command line arguments <tt>nx, ny, nz, num_iter</tt> specify the size of the computational domain as well as the number of iterations.

In [15]:
srun -n 12 ./stencil_2d.x+orig -nx 128 -ny 128 -nz 64 -num_iter 1024

 Counted floating-point operations [GFLOP] =  1.9045257568359375
 Counted memory transfers [GB] =  111.0479736328125
 --------------------------------------------------------------------------
  Timers:
   number of workers =   12
 --------------------------------------------------------------------------
  Id      Tag                     #calls        min[s]        max[s]       mean[s] 
   1      work                         1        8.6135        8.6885        8.6457
 --------------------------------------------------------------------------


<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>3.</b> Compute the arithmetic intensity $I$ of our stencil program using the flop and byte counters you have introduced.<br>
<b>4.</b> Compute the performance in GFLOP/s of our stencil program. Is our program memory bound or compute bound? Which % of peak FLOP/s and memory bandwidth do we achieve?<br>
<b>5.</b> Run the stencil program for different <tt>nx</tt> and <tt>ny</tt> (see below). Import the data into a Python notebook and make a loglog-plot of the working set size $n = nx \times ny \times nz \times 3$ in MB versus the runtime per gridpoint $r = \mathrm{runtime} \, / \, nx \, / \, ny \, / nz$ in $\mu s$. What would you expect for a von Neumann architecture? What might be the reason that the behavior is different?
</div>

In [23]:
echo "# nx ny nz perf[GFLOP] mem[GB] runtime[s]"
echo "data = np.array( [ \\"
nz=64
for nx in 16 32 48 64 96 128 ; do
for ny in 16 32 48 64 96 128 ; do
    sleep 2
    srun -n 12 ./stencil_2d.x+orig -nx ${nx} -ny ${ny} -nz ${nz} -num_iter 1024 2>&1 1> /tmp/stencil_2d.$$.out
    gflop=`cat /tmp/stencil_2d.$$.out | grep "GFLOP" | awk '{print $6}' | tr -d '\n'`
    gbyte=`cat /tmp/stencil_2d.$$.out | grep "GB" | awk '{print $6}' | tr -d '\n'`
    runtime=`cat /tmp/stencil_2d.$$.out | grep " work " | awk '{print $6}' | tr -d '\n'`
    echo "[${nx}, ${ny}, ${nz}, ${gflop}, ${gbyte}, ${runtime}], \\"
    /bin/rm -f /tmp/stencil_2d.*.out 2>/dev/null 1>/dev/null
done
done
echo "] )"

# nx ny nz perf[GFLOP] mem[GB] runtime[s]
data = np.array( [ \
[16, 16, 64, 3.31878662109375E-2, 2.4078369140625, 0.0330], \
[16, 32, 64, 6.43157958984375E-2, 4.4307861328125, 0.0671], \
[16, 48, 64, 9.54437255859375E-2, 6.4537353515625, 0.1007], \
[16, 64, 64, 0.1265716552734375, 8.4766845703125, 0.1386], \
[16, 96, 64, 0.1888275146484375, 12.5225830078125, 0.2068], \
[16, 128, 64, 0.2510833740234375, 16.5684814453125, 0.2740], \
[32, 16, 64, 6.43157958984375E-2, 4.4307861328125, 0.0558], \
[32, 32, 64, 0.1247406005859375, 8.0928955078125, 0.1071], \
[32, 48, 64, 0.1851654052734375, 11.7550048828125, 0.1620], \
[32, 64, 64, 0.2455902099609375, 15.4171142578125, 0.2145], \
[32, 96, 64, 0.3664398193359375, 22.7413330078125, 0.5284], \
[32, 128, 64, 0.4872894287109375, 30.0655517578125, 1.7166], \
[48, 16, 64, 9.54437255859375E-2, 6.4537353515625, 0.0764], \
[48, 32, 64, 0.1851654052734375, 11.7550048828125, 0.1475], \
[48, 48, 64, 0.2748870849609375, 17.0562744140625, 0.2206], \
[48, 64

![runtime_vs_size](img/runtime_vs_size.png)

## Performance Analysis Tool (perftool-lite)

We can also use a performance analysis tool from Cray named <tt>perftools-lite</tt> to analyze the performance of our stencil program. In fact, our program has already been compiled for performance analysis with <tt>perftools-lite</tt> since we have loaded the corresponding module. <tt>stencil_2d.x+orig</tt> is the original executable without instrumentation for performance analysis and <tt>stencil_2d.x</tt> is an executable specifically prepared for performance analysis.

<div class="alert alert-block alert-info">
<b>Now it's your turn...</b><br>
<b>6.</b> Run your program with <tt>perftools-lite</tt>. Read the report generated by <tt>perftools-lite</tt> carefully. What can you learn from the profiling?<br>
<b>7.</b> Compare the memory transfer numbers against your values. By which factor are you off? What could be the reason?<br>
<b>8.</b> Run again with $nx = 64$ and $ny = 32$. Did the factor by which you are off change? Is this consistent with the findings above?<br>
</div>

In [17]:
srun -n 12 ./stencil_2d.x -nx 128 -ny 128 -nz 64 -num_iter 1024

CrayPat/X:  Version 7.1.1 Revision 7c0ddd79b  08/19/19 16:58:46
 Counted floating-point operations [GFLOP] =  1.9045257568359375
 Counted memory transfers [GB] =  111.0479736328125
 --------------------------------------------------------------------------
  Timers:
   number of workers =   12
 --------------------------------------------------------------------------
  Id      Tag                     #calls        min[s]        max[s]       mean[s] 
   1      work                         1        8.6089        8.7020        8.6579
 --------------------------------------------------------------------------

#################################################################
#                                                               #
#            CrayPat-lite Performance Statistics                #
#                                                               #
#################################################################

CrayPat/X:  Version 7.1.1 Revision 7c0ddd79b  08/19/19

In [19]:
srun -n 12 ./stencil_2d.x -nx 64 -ny 32 -nz 64 -num_iter 1024

CrayPat/X:  Version 7.1.1 Revision 7c0ddd79b  08/19/19 16:58:46
 Counted floating-point operations [GFLOP] =  0.2455902099609375
 Counted memory transfers [GB] =  15.4171142578125
 --------------------------------------------------------------------------
  Timers:
   number of workers =   12
 --------------------------------------------------------------------------
  Id      Tag                     #calls        min[s]        max[s]       mean[s] 
   1      work                         1        0.1811        0.1892        0.1852
 --------------------------------------------------------------------------

#################################################################
#                                                               #
#            CrayPat-lite Performance Statistics                #
#                                                               #
#################################################################

CrayPat/X:  Version 7.1.1 Revision 7c0ddd79b  08/19/19 